In [13]:
"""
Script 2: AutoRectifier

Description:
This script rectifies a batch of raster images (e.g., scanned and preprocessed micro-section drawings)
by applying a scale transformation based on a 1-meter vector scale stored in a shapefile. Each image is 
resized accordingly and georeferenced as a standalone GeoTIFF using a synthetic coordinate system with 
systematic offsets for spatial arrangement. No GCPs are required.

To use this script correctly, the user must define user settings (see below):

Notes:
- The script automatically calculates the scale factor and pixel resolution.
- Output rasters are saved as single-band (grayscale) GeoTIFFs with spatial referencing.
- Ensure the vector scale shapefile contains a valid LineString as its first geometry.

"""

import numpy as np
import cv2
from matplotlib import pyplot as plt
import geopandas as gpd
from shapely.geometry import LineString
import os
import time
import rasterio
from rasterio.transform import from_origin

# ---------------------------------------------
# USER SETTINGS (EDIT BEFORE RUNNING)
# ---------------------------------------------
input_folder = r"C:\path_to_input_folder"  # folder with input images
output_folder = r"C:\path_to_output_folder"  # folder for output GeoTIFFs
shapefile_path = r"C:\path_to_SCALE.shp"  # shapefile with vector scale (1m line)
crs = "EPSG:XXX"  # Coordinate Reference System to apply to all outputs
initial_upper_left_x = 0  # upper left X of first raster
initial_upper_left_y = 0  # upper left Y of first raster
offset = 1.5  # spacing in meters between rasters (used to calculate each position)
columns_per_row = 25  # how many rasters per row before moving to new line
image_scale_length_pixels = 744  # length of 1 meter scale bar in pixels on the image
real_world_length_meters = 1.000366  # length of the scale in meters (should match vector length)

# ---------------------------------------------
# FUNCTIONS
# ---------------------------------------------
def display(img, title=''):
    """Display a grayscale image with matplotlib."""
    plt.imshow(img, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

def get_line_length_from_shapefile(shapefile_path):
    """Extracts the length of the first LineString in a shapefile."""
    gdf = gpd.read_file(shapefile_path)
    line_geom = gdf.geometry.iloc[0]
    if isinstance(line_geom, LineString):
        return line_geom.length
    else:
        raise ValueError("Shapefile does not contain a LineString geometry.")

def calculate_scale_factor(vector_length_meters, image_scale_length_pixels, resolution_meters_per_pixel):
    """Calculates the scaling factor to apply to image based on vector length."""
    vector_length_pixels = vector_length_meters / resolution_meters_per_pixel
    scale_factor = image_scale_length_pixels / vector_length_pixels
    return scale_factor

def rectify_image_without_gcp(img, scale_factor):
    """Resizes the image based on a scale factor (no GCPs used)."""
    height, width = img.shape[:2]
    new_size = (int(width * scale_factor), int(height * scale_factor))
    resized_img = cv2.resize(img, new_size, interpolation=cv2.INTER_LINEAR)
    return resized_img

# ---------------------------------------------
# MAIN PROCESS
# ---------------------------------------------
start_time = time.time()

# Get scale from shapefile
meritko_length_meters = get_line_length_from_shapefile(shapefile_path)
print("Vector scale length (m):", meritko_length_meters)

# Calculate pixel resolution and scale factor
resolution_meters_per_pixel = real_world_length_meters / image_scale_length_pixels
print("Resolution (meters per pixel):", resolution_meters_per_pixel)

scale_factor = calculate_scale_factor(meritko_length_meters, image_scale_length_pixels, resolution_meters_per_pixel)
print("Scale factor:", scale_factor)

# Create output folder if needed
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Iterate over images
for i, filename in enumerate(os.listdir(input_folder)):
    if filename.lower().endswith((".png", ".jpg", ".jpeg")):
        input_path = os.path.join(input_folder, filename)
        img = cv2.imread(input_path, 0)
        print(f"Processing {filename} with original dimensions: {img.shape}")

        # Rescale image
        warped_img = rectify_image_without_gcp(img, scale_factor)
        print(f"Resized dimensions for {filename}: {warped_img.shape}")

        # Calculate upper-left coordinate for this raster
        col_number = i % columns_per_row
        row_number = i // columns_per_row
        upper_left_x = initial_upper_left_x + col_number * offset
        upper_left_y = initial_upper_left_y - row_number * offset
        print(f"Upper-left corner for {filename}: X = {upper_left_x}, Y = {upper_left_y}")

        # Create rasterio transform
        transform = from_origin(upper_left_x, upper_left_y, resolution_meters_per_pixel, resolution_meters_per_pixel)

        # Prepare image array for rasterio
        warped_array = np.expand_dims(warped_img, axis=0)  # shape (1, H, W)

        # Save to GeoTIFF
        geo_output_path = os.path.join(output_folder, filename.replace(".png", ".tif").replace(".jpg", ".tif"))
        try:
            with rasterio.open(
                geo_output_path, "w",
                driver="GTiff",
                height=warped_array.shape[1],
                width=warped_array.shape[2],
                count=1,
                dtype=warped_array.dtype,
                crs=crs,
                transform=transform
            ) as dst:
                dst.write(warped_array)
            print(f"GeoTIFF saved: {geo_output_path}")
        except Exception as e:
            print(f"Error writing {filename}: {e}")

# Report time
elapsed_time = time.time() - start_time
print(f"All images rectified in {elapsed_time:.2f} seconds.")

Vector scale length (m): 0.9992981185365547
Resolution (meters per pixel): 0.0013445779569892474
Scale factor: 1.0010686315160977
Processing CS_10.jpg with original dimensions: (2338, 1653)
Resized dimensions for CS_10.jpg: (2340, 1654)
Upper-left corner for CS_10.jpg: X = 0.0, Y = 0.0
GeoTIFF saved: C:\\Users\\filip\\Documents\\ARC_GIS_Dukovany\\test_rectified\CS_10.tif
Processing CS_100.jpg with original dimensions: (2338, 1653)
Resized dimensions for CS_100.jpg: (2340, 1654)
Upper-left corner for CS_100.jpg: X = 1.5, Y = 0.0
GeoTIFF saved: C:\\Users\\filip\\Documents\\ARC_GIS_Dukovany\\test_rectified\CS_100.tif
Processing CS_1013.jpg with original dimensions: (2338, 1653)
Resized dimensions for CS_1013.jpg: (2340, 1654)
Upper-left corner for CS_1013.jpg: X = 3.0, Y = 0.0
GeoTIFF saved: C:\\Users\\filip\\Documents\\ARC_GIS_Dukovany\\test_rectified\CS_1013.tif
Processing CS_1014.jpg with original dimensions: (2338, 1653)
Resized dimensions for CS_1014.jpg: (2340, 1654)
Upper-left corn